Load the dataset

In [3]:
import pandas as pd
df = pd.read_csv("Cleaned_Ecommerce_Dataset.csv")
print( df.shape)

(51290, 21)


Feature selection & feature engineering

In [5]:
import numpy as np
drop_cols = ['Order_Date', 'Time', 'Customer_Id']
for c in drop_cols:
    if c in df.columns:
        df = df.drop(columns=c)

if 'Profit_Margin' not in df.columns:
    df['Profit_Margin'] = df['Profit'] / df['Sales']

if 'Revenue_per_Unit' not in df.columns:
    df['Revenue_per_Unit'] = df['Sales'] / df['Quantity'].replace(0, np.nan)
    df['Revenue_per_Unit'] = df['Revenue_per_Unit'].fillna(df['Sales'])


 Numeric pipeline: impute + scale

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

 Categorical pipeline: impute + one-hot

In [12]:
from sklearn.preprocessing import OneHotEncoder
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

In [14]:
from sklearn.compose import ColumnTransformer
TARGETS = ['Sales', 'Profit']
all_cols = df.columns.tolist()
feature_candidates = [c for c in all_cols if c not in TARGETS]
num_feats = [c for c in feature_candidates if pd.api.types.is_numeric_dtype(df[c])]
cat_feats = [c for c in feature_candidates if pd.api.types.is_object_dtype(df[c]) or pd.api.types.is_categorical_dtype(df[c])]
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_feats),
    ('cat', cat_transformer, cat_feats)
])

C:\Users\palak\AppData\Local\Temp\ipykernel_14092\2156119931.py:6: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_feats = [c for c in feature_candidates if pd.api.types.is_object_dtype(df[c]) or pd.api.types.is_categorical_dtype(df[c])]


train & split code

In [16]:
X = df[num_feats + cat_feats]
y = df[TARGETS]
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)


Train shape: (41032, 18) (41032, 2)
Test shape: (10258, 18) (10258, 2)
